In [1]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

model_id = "codellama/CodeLlama-34b-hf"

peft_model_id = "/home/ksaff/Desktop/ttyd/fine_tuning/2nd_try"
config = PeftConfig.from_pretrained(peft_model_id)

bnb_config = BitsAndBytesConfig(
                                load_in_4bit=True,
                                bnb_4bit_use_double_quant=True,
                                bnb_4bit_quant_type="nf4",
                                bnb_4bit_compute_dtype=torch.bfloat16
                                )

model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    return_dict=True,
    quantization_config=bnb_config, device_map={'': 0}
)
tokenizer = AutoTokenizer.from_pretrained(
                                          config.base_model_name_or_path,
                                          )

/home/ksaff/miniconda3/envs/llm_jupyter/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


/home/ksaff/.cache/huggingface/hub/models--codellama--CodeLlama-34b-hf/snapshots/82128714b6174570a64b3dd1f3e9c146bda26cf9/model.safetensors.index.json
['/home/ksaff/.cache/huggingface/hub/models--codellama--CodeLlama-34b-hf/snapshots/82128714b6174570a64b3dd1f3e9c146bda26cf9/model-00001-of-00007.safetensors', '/home/ksaff/.cache/huggingface/hub/models--codellama--CodeLlama-34b-hf/snapshots/82128714b6174570a64b3dd1f3e9c146bda26cf9/model-00002-of-00007.safetensors', '/home/ksaff/.cache/huggingface/hub/models--codellama--CodeLlama-34b-hf/snapshots/82128714b6174570a64b3dd1f3e9c146bda26cf9/model-00003-of-00007.safetensors', '/home/ksaff/.cache/huggingface/hub/models--codellama--CodeLlama-34b-hf/snapshots/82128714b6174570a64b3dd1f3e9c146bda26cf9/model-00004-of-00007.safetensors', '/home/ksaff/.cache/huggingface/hub/models--codellama--CodeLlama-34b-hf/snapshots/82128714b6174570a64b3dd1f3e9c146bda26cf9/model-00005-of-00007.safetensors', '/home/ksaff/.cache/huggingface/hub/models--codellama--Cod

Loading checkpoint shards:  14%|█▍        | 1/7 [00:03<00:21,  3.56s/it]


KeyboardInterrupt: 

In [4]:
# model_name = "codellama/CodeLlama-34b-hf"
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForCausalLM.from_pretrained(
#     model_name,
#     trust_remote_code=True,
#     quantization_config=bnb_config,
#     device_map={'': 0},
#     use_cache=True,
# )

ImportError: Using `load_in_8bit=True` requires Accelerate: `pip install accelerate` and the latest version of bitsandbytes `pip install -i https://test.pypi.org/simple/ bitsandbytes` or pip install bitsandbytes` 

In [52]:
database_scheme = """
CREATE TABLE Salaries
  Id INTEGER PRIMARY KEY, -- Unique ID for each employee
  EmployeeName VARCHAR, -- Name of the employee
  JobTitle VARCHAR, -- Name of employees proffesion
  BasePay NUMERIC, -- Base pay of employee
  OvertimePay NUMERIC, -- Overtime pay of employee
  OtherPay NUMERIC, -- Other pays of employee
  Benefits NUMERIC, -- Benefits of employee
  TotalPay NUMERIC, -- Total pay of employee
  TotalPayBenefits NUMERIC, -- Sum of pay benefits of employee
  Year INTEGER, -- Year data from row reffers to
"""

question = "Calculate the Total Pay Including Benefits for Each Employee"

text = (
    """### Task
    Generate a SQL query to answer the following question:
    {question}
### Database Schema
    This query will run on a database whose schema is represented in this string:
    {database_scheme}
### SQL
    Given the database schema, here is the SQL query that answers `{question}`:
    ```sql
    """.format(question=question, database_scheme=database_scheme)
)

In [53]:
eos_token_id = tokenizer.eos_token_id
input_ids = tokenizer.encode(text, return_tensors='pt')
input_ids = input_ids.to('cuda')

In [54]:
inputs = {
    'input_ids': input_ids,
    'max_length': 400,
    'max_time': 7,
}

output = model.generate(**inputs)

generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(generated_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### Task
    Generate a SQL query to answer the following question:
    Calculate the Total Pay Including Benefits for Each Employee
### Database Schema
    This query will run on a database whose schema is represented in this string:
    
CREATE TABLE Salaries
  Id INTEGER PRIMARY KEY, -- Unique ID for each employee
  EmployeeName VARCHAR, -- Name of the employee
  JobTitle VARCHAR, -- Name of employees proffesion
  BasePay NUMERIC, -- Base pay of employee
  OvertimePay NUMERIC, -- Overtime pay of employee
  OtherPay NUMERIC, -- Other pays of employee
  Benefits NUMERIC, -- Benefits of employee
  TotalPay NUMERIC, -- Total pay of employee
  TotalPayBenefits NUMERIC, -- Sum of pay benefits of employee
  Year INTEGER, -- Year data from row reffers to

### SQL
    Given the database schema, here is the SQL query that answers `Calculate the Total Pay Including Benefits for Each Employee`:
    ```sql
     SELECT EmployeeName, SUM(TotalPayBenefits) AS TotalPayBenefits
     FROM Salaries
   

In [55]:
import sqlparse
query = sqlparse.format(generated_text.split("```sql")[-1], reindent=True)[:-3]

In [56]:
print(query)


SELECT EmployeeName,
       SUM(TotalPayBenefits) AS TotalPayBenefits
FROM Salaries
GROUP BY EmployeeName
ORDER BY TotalPayBenefits DESC 


In [57]:
import sqlite3
from dotenv import load_dotenv
load_dotenv()

conn = sqlite3.connect('/home/ksaff/Desktop/sql/choosen_DB.sqlite')
c = conn.cursor()

def execute_query(query):

    c.execute(query)
    result = c.fetchall()

    # Extract column names
    columns = [desc[0] for desc in c.description]
    
    # Prepare the response
    response = {"query": query, "headers": columns, "data": result}

    return response

In [58]:
result = execute_query(query)
# print(f'User instruction: {instruction}')
# print(f'model output: {response}')
'Result query:', result['data']

('Result query:',
 [('Kevin Lee', 1778487.17),
  ('Steven Lee', 1706550.7600000002),
  ('William Wong', 1459954.17),
  ('Stanley Lee', 1457279.6800000002),
  ('Richard Lee', 1268668.84),
  ('John L Martin', 1054756.22),
  ('Wayne Wong', 1021109.55),
  ('Alan Wong', 1011610.5800000001),
  ('John Chan', 946239.26),
  ('Michael Brown', 907786.4800000001),
  ('Robert Wong', 901466.1600000001),
  ('John Miller', 874900.7300000001),
  ('Amy P Hart', 863398.99),
  ('David Shinn', 844056.71),
  ('Gregory P Suhr', 843834.5),
  ('Joanne M Hayes-White', 839788.5),
  ('Thomas Smith', 831608.5700000001),
  ('Ryan J Jones', 824661.3899999999),
  ('Frank Lee', 823942.97),
  ('Ellen G Moffatt', 817504.8200000001),
  ('KEVIN LEE', 809424.8400000001),
  ('John Lee', 799287.73),
  ('David L Franklin', 799045.86),
  ('Ana Gonzalez', 795145.15),
  ('William Lee', 794871.35),
  ('Harlan L Kelly-Jr', 794616.56),
  ('Michael Lee', 792816.3799999999),
  ('Michael Wong', 787388.0799999998),
  ('Henry Yee', 7854